<a href="https://colab.research.google.com/github/OneFineStarstuff/Cosmic-Brilliance/blob/main/Logging%2C_Visualisation_%26_Statistical_Tests.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# === cell 9: experiment logger ===
import pathlib
import datetime
import json
import pandas as pd
from torch.utils.tensorboard import SummaryWriter

# Create a timestamped run directory
logdir = pathlib.Path("runs") / datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
logdir.mkdir(parents=True, exist_ok=True)

# TensorBoard writer
writer = SummaryWriter(log_dir=logdir.as_posix())

# In-memory log for quick analysis (optional)
episode_log = []

def log_episode(ep_idx: int, reward: float, length: int, safety_cost: float):
    """
    Logs a single episode's metrics to TensorBoard, JSONL file, and in-memory list.

    Args:
        ep_idx (int): Episode index.
        reward (float): Episode reward.
        length (int): Episode length (steps).
        safety_cost (float): Safety cost metric.
    """
    row = {
        "episode": ep_idx,
        "reward": reward,
        "length": length,
        "safety": safety_cost
    }

    # TensorBoard scalars
    writer.add_scalar("reward/episode", reward, ep_idx)
    writer.add_scalar("safety/episode", safety_cost, ep_idx)
    writer.flush()  # ensure data is written to disk

    # Append to JSONL file
    with open(logdir / "episodes.jsonl", "a") as f:
        f.write(json.dumps(row) + "\n")

    # Optional: keep in-memory DataFrame
    episode_log.append(row)

def get_episode_df() -> pd.DataFrame:
    """
    Returns all logged episodes as a Pandas DataFrame.
    """
    return pd.DataFrame(episode_log)

def save_episode_csv(filename: str = "episodes.csv"):
    """
    Saves all logged episodes to a CSV file in the log directory.
    """
    df = get_episode_df()
    csv_path = logdir / filename
    df.to_csv(csv_path, index=False)
    print(f"Episode log saved to {csv_path}")

# Example usage:
# for ep in range(10):
#     log_episode(ep, reward=ep*1.5, length=100+ep, safety_cost=ep*0.1)
# save_episode_csv()